In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [183]:
with open("dataset/breast-cancer.labels", "r") as f:
    labels = [line.strip() for line in f.readlines()]
train_data = pd.io.parsers.read_csv("dataset/breast-cancer-train.dat", names=labels)
validate_data = pd.io.parsers.read_csv("dataset/breast-cancer-validate.dat", names=labels)
train_data_malignant = train_data[train_data["Malignant/Benign"] == "M"]
train_data_benign = train_data[train_data["Malignant/Benign"] == "B"]

In [184]:
def least_square():
    linear_train = train_data.drop(["patient ID", "Malignant/Benign"], axis=1).values
    linear_validate = validate_data.drop(["patient ID", "Malignant/Benign"], axis=1).values

    def create_quadratic_representation(data):
        df = data.copy()
        for i in range(len(quad_columns)):
            df[f"{i}^2"] = data[quad_columns[i]] ** 2
        for i in range(len(quad_columns)):
            for j in range(i + 1, len(quad_columns)):
                df[f"{i}_{j}"] = data[quad_columns[i]] * data[quad_columns[j]]
        return df.values

    quad_columns = ["radius (mean)", "perimeter (mean)", "area (mean)", "symmetry (mean)"]
    quadratic_train = create_quadratic_representation(train_data[quad_columns])
    quadratic_validate = create_quadratic_representation(validate_data[quad_columns])

    b_training = np.where(train_data[['Malignant/Benign']] == "M", 1, -1)
    b_validate = np.where(validate_data[['Malignant/Benign']] == "M", 1, -1)

    cov_mat_lin = linear_train.T @ linear_train
    cov_mat_quad = quadratic_train.T @ quadratic_train

    weights_linear = np.linalg.solve(cov_mat_lin, linear_train.T @ b_training)
    weights_quadratic = np.linalg.solve(cov_mat_quad, quadratic_train.T @ b_training)

    λ = 0.01
    svd = scipy.linalg.lstsq(cov_mat_lin + λ * np.eye(cov_mat_lin.shape[0]), linear_train.T @ b_training)

    p_lin = linear_validate @ weights_linear
    p_quad = quadratic_validate @ weights_quadratic
    p_lambda = linear_validate @ weights_linear
    p_svd = linear_validate @ svd[0]
    def calc_acc(p_vec, b_vec):
        tp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b > 0])
        tn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b < 0])
        fp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b <= 0])
        fn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b > 0])
        return int(tp), int(tn), int(fp), int(fn), float((tp + tn) / (tp + tn + fp + fn))

    tp_lin, tn_lin, fp_lin, fn_lin, acc_lin = calc_acc(p_lin, b_validate)
    tp_quad, tn_quad, fp_quad, fn_quad, acc_quad = calc_acc(p_quad, b_validate)
    tp_lambda, tn_lambda, fp_lambda, fn_lambda, acc_lambda = calc_acc(p_lambda, b_validate)
    tp_svd, tn_svd, fp_svd, fn_svd, acc_svd = calc_acc(p_svd, b_validate)
    return (tp_lin, tn_lin, fp_lin, fn_lin, acc_lin), (tp_quad, tn_quad, fp_quad, fn_quad, acc_quad), (tp_lambda, tn_lambda, fp_lambda, fn_lambda, acc_lambda),(tp_svd, tn_svd, fp_svd, fn_svd, acc_svd)

In [185]:
def gradient_descent(eps=0.0001, n_iterations=100000):
    A = train_data.drop(["patient ID", "Malignant/Benign"], axis=1).values
    A = np.c_[np.ones(A.shape[0]), A]

    A_validate = validate_data.drop(["patient ID", "Malignant/Benign"], axis=1).values
    A_validate = np.c_[np.ones(A_validate.shape[0]), A_validate]

    b = np.where(train_data[['Malignant/Benign']] == "M", 1, -1)
    b_validate = np.where(validate_data[['Malignant/Benign']] == "M", 1, -1)
    
    b = b.flatten()

    AtA = A.T @ A

    eigv = scipy.linalg.eigvals(AtA)
    real_eigv = np.real(eigv)

    lambda_max = np.max(real_eigv)
    lambda_min = np.min(real_eigv)

    alpha = 1 / (lambda_max + lambda_min)
    
    def calc_gradient(A, x, b):
        return 2 * A.T @ (A @ x - b)
    
    n_samples, n_features = A.shape
    
    x = np.zeros(n_features)
    
    k = 1
    
    while k < n_iterations:
        x_next = x - alpha * calc_gradient(A, x, b)
        if np.linalg.norm(x_next - x) < eps:
            break
        x = x_next
        k += 1
        
    p = A_validate @ x

    def calc_acc(p_vec, b_vec):
        tp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b > 0])
        tn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b < 0])
        fp = np.sum([1 for p, b in zip(p_vec, b_vec) if p > 0 and b <= 0])
        fn = np.sum([1 for p, b in zip(p_vec, b_vec) if p <= 0 and b > 0])
        total = tp + tn + fp + fn
        accuracy = float((tp + tn) / total) if total > 0 else 0.0
        return int(tp), int(tn), int(fp), int(fn), accuracy

    tp, tn, fp, fn, accuracy = calc_acc(p, b_validate.flatten())

    return (tp, tn, fp, fn, accuracy), k

In [186]:
least_square()

((58, 194, 6, 2, 0.9692307692307692),
 (55, 185, 15, 5, 0.9230769230769231),
 (58, 194, 6, 2, 0.9692307692307692),
 (55, 199, 1, 5, 0.9769230769230769))

In [187]:
grad_result, grad_iterations = gradient_descent()
grad_result, grad_iterations

((58, 177, 23, 2, 0.9038461538461539), 100000)